In [17]:
import pandas as pd
import re
import jellyfish
pd.set_option('display.precision',10)
from pyjarowinkler import distance
df_npa = pd.read_csv(r'C:\Users\KC\Desktop\EssilorWP\NPA_ForFuzzy.csv')
df_macola = pd.read_csv(r"C:\Users\KC\Desktop\WhitespaceProject\Macola.csv")
df_macola['Phone'] = df_macola['Phone'].astype(str)          
df_macola['Phone'] = df_macola['Phone'].str.replace('.0', ' ')
df_macola['Practice Phone'] = df_macola['Practice Phone'].astype(str)          
df_macola['Practice Phone'] = df_macola['Practice Phone'].str.replace('.0', ' ')
df_npa['Provider Business Mailing Address Telephone Number'] = df_npa['Provider Business Mailing Address Telephone Number'].astype(str)          
df_npa['Provider Business Mailing Address Telephone Number'] = df_npa['Provider Business Mailing Address Telephone Number'].str.replace('.0', ' ')
df_npa['Provider Business Practice Location Address Telephone Number'] =df_npa['Provider Business Practice Location Address Telephone Number'].astype(str)          
df_npa['Provider Business Practice Location Address Telephone Number'] =df_npa['Provider Business Practice Location Address Telephone Number'].str.replace('.0', ' ')

C:\Users\KC\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
df_npa.set_index('Unnamed: 0', inplace=True)
df_npa.drop(['check1','check2'],axis=1)

,NPI,Entity Type Code,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,...,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Is Sole Proprietor,Is Organization Subpart,Parent Organization LBN,Parent Organization TIN,Authorized Official Name Prefix Text,Authorized Official Name Suffix Text,Authorized Official Credential Text
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2317935,1003000217,2.0,<UNAVAIL>,"TRI-STATE EYE CARE CENTER, LTD.",NaN,NaN,NaN,NaN,NaN,NaN,...,L,OWNER,3045234819.0,NaN,N,NaN,NaN,DR.,NaN,OD
2341457,1003002155,2.0,<UNAVAIL>,HARVARD SQUARE EYE CARE P.C.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,PRESIDENT,61733545590.0,NaN,N,NaN,NaN,DR.,NaN,O.D.
2340044,1003002296,2.0,<UNAVAIL>,"DUPUIS OPTOMETRY, INC.",NaN,NaN,NaN,NaN,NaN,NaN,...,P,PRESIDENT/OWNER,3034492401.0,NaN,N,NaN,NaN,DR.,NaN,O.D.
2338029,1003002445,2.0,<UNAVAIL>,BETHEL VISION CENTER,NaN,NaN,NaN,NaN,NaN,NaN,...,D,OWNER,6144594093.0,NaN,N,NaN,NaN,DR.,NaN,O.D.
2334946,1003002759,2.0,<UNAVAIL>,MOBILE PHYSICIAN SERVICES INC.,NaN,NaN,NaN,NaN,NaN,NaN,...,M,MEDICAL DIRECTOR,8552320644.0,NaN,N,NaN,NaN,DR.,NaN,M.D.
2352889,1003003096,2.0,<UNAVAIL>,"JOSEPH G. HEINRICH, O.D., A.P.C.",NaN,NaN,NaN,NaN,NaN,NaN,...,G.,OPTOMETRIST,9496613669.0,NaN,N,NaN,NaN,DR.,NaN,O.D.
2355914,1003004813,2.0,<UNAVAIL>,"PROEYE GROUP GATEWAY, P.C.",NaN,NaN,NaN,NaN,NaN,NaN,...,JEFFREY,OPTOMETRIST/OWNER,4024673423.0,NaN,N,NaN,NaN,DR.,NaN,O.D.
2371301,1003005356,2.0,<UNAVAIL>,A BETTER VUE EYE PHYSICIANS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,MANAGING MEMBER,2392622020.0,NaN,N,NaN,NaN,DR.,NaN,M.D.
2370861,1003005562,2.0,<UNAVAIL>,"HERITAGE EYE CARE, INC",NaN,NaN,NaN,NaN,NaN,NaN,...,DANIEL,OWNER,9194531220.0,NaN,N,NaN,NaN,DR.,III,O.D.


In [19]:
df_npa = df_npa.sort_values("NPI")
cols_practice_npa = ['Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name']
cols_practice_macola = ['Practice Street Address 1', 'Practice Street Address 2','Practice City']
df_macola['check1'] = df_macola[cols_practice_macola].apply(lambda x: ','.join(x.dropna()), axis=1)
df_npa['check1'] = df_npa[cols_practice_npa].apply(lambda x: ','.join(x.dropna()), axis=1)
cols_practice_npa = ['Provider First Line Business Mailing Address','Provider Second Line Business Mailing Address','Provider Business Mailing Address City Name']
cols_practice_macola = ['Shipping Address Line 1', 'Shipping Address Line 2', 'Shipping Address Line 3','Shipping City']
df_macola['check2'] = df_macola[cols_practice_macola].apply(lambda x: ','.join(x.dropna()), axis=1)
df_npa['check2'] = df_npa[cols_practice_npa].apply(lambda x: ','.join(x.dropna()), axis=1)

In [4]:
businesspractices_npa = df_npa['Provider Business Practice Location Address Telephone Number'].tolist()
print(len(businesspractices_npa))
businesspractices_npa = list(set(businesspractices_npa))
print(len(businesspractices_npa))

42030
38380


In [5]:
filter = df_macola["check1"] != ""
df_macola1 = df_macola[filter]
addressenrichfilter = df_macola["check1"] == ""
df_macola_toenrich = df_macola[addressenrichfilter]
df_macola_toenrich.shape

(11761, 28)

In [6]:
businesspractices_macola = df_macola['Practice Phone'].tolist()
print(len(businesspractices_macola))
businesspractices_macola = list(set(businesspractices_macola))
print(len(businesspractices_macola))

48747
32604


In [7]:
filter = df_macola["check1"] != ""
df_macola1 = df_macola[filter]


In [8]:
df_macola1 = df_macola1.dropna(subset=['check1'])
df_macola1 = df_macola1.dropna(subset=['Practice Phone'])

In [9]:
f = open(r"C:\Users\KC\Desktop\EssilorWP\fuzzysearchvaluesPA.txt", 'w')
index_jf = []
businesspracticephone = df_macola1['Practice Phone'].tolist()
for index,row in df_npa.iterrows():
    if row['Provider Business Practice Location Address Telephone Number'] in businesspracticephone:
        f1 = re.sub('[^a-zA-Z0-9]', '', row['check1'])
        checkvalue = df_macola1.loc[df_macola1['Practice Phone'] == row['Provider Business Practice Location Address Telephone Number'], 'check1'].values
        if len(checkvalue)>0:
            f2 = re.sub('[^a-zA-Z0-9]', '', checkvalue[0])
            if jellyfish.jaro_winkler(f1,f2)>.86:
                index_jf.append(index)   
                f = open(r"C:\Users\KC\Desktop\EssilorWP\fuzzysearchvaluesPA.txt", 'a')
                f.write(str(index) + ',' + f1 + ',' + f2 + '\n')
                f.close 
print(len(index_jf))

12707


In [10]:
fuzzindex = []
fuzzdata = pd.read_csv(r"C:\Users\KC\Desktop\EssilorWP\fuzzysearchvaluesPA.txt", header = None)
fuzzdata.columns = ['a','b','c']

In [11]:
for index,row in fuzzdata.iterrows():
    if str(row.b[0:8]) == str(row.c[0:8]):
        if len(re.findall(r'\d+', row.b))>=2 & len(re.findall(r'\d+', row.c))>=2:
            if (re.findall(r'\d+', row.b)[1] == re.findall(r'\d+', row.c)[1]):
                fuzzindex.append(row.a)
                f = open(r"C:\Users\KC\Desktop\EssilorWP\ram2fuzzyvalues-11.txt", 'a')
                f.write(str(row.a) + ',' + str(row.b) + ',' + str(row.c) + '\n')
                f.close 
        if len(re.findall(r'\d+', row.b))==1 & len(re.findall(r'\d+', row.c))==1:
            if (re.findall(r'\d+', row.b)[0] == re.findall(r'\d+', row.c)[0]):
                fuzzindex.append(row.a)
                f = open(r"C:\Users\KC\Desktop\EssilorWP\ram2fuzzyvalues-11.txt", 'a')
                f.write(str(row.a) + ',' + str(row.b) + ',' + str(row.c) + '\n')
                f.close
                
        if len(re.findall(r'\d+', row.b))==1 & len(re.findall(r'\d+', row.c))>1:
            if (re.findall(r'\d+', row.b)[0] == re.findall(r'\d+', row.c)[0]):
                fuzzindex.append(row.a)
                f = open(r"C:\Users\KC\Desktop\EssilorWP\ram2fuzzyvalues-11.txt", 'a')
                f.write(str(row.a) + ',' + str(row.b) + ',' + str(row.c) + '\n')
                f.close
                
        if len(re.findall(r'\d+', row.b))>1 & len(re.findall(r'\d+', row.c))==1:
            if (re.findall(r'\d+', row.b)[0] == re.findall(r'\d+', row.c)[0]):
                fuzzindex.append(row.a)
                f = open(r"C:\Users\KC\Desktop\EssilorWP\ram2fuzzyvalues-11.txt", 'a')
                f.write(str(row.a) + ',' + str(row.b) + ',' + str(row.c) + '\n')
                f.close
print(len(fuzzindex))

11378


In [12]:
len(set(index_jf))

12707

In [13]:
businesslegal_npa = df_macola['Phone'].tolist()
print(len(businesslegal_npa))
businesslegal_npa = list(set(businesspractices_npa))
print(len(businesslegal_npa))

f = open(r"C:\Users\KC\Desktop\EssilorWP\fuzzysearchvaluesBA.txt", 'w')
index_lf = []
for index,row in df_npa.iterrows():
    if row['Provider Business Mailing Address Telephone Number'] in businesslegal_npa:
        f1 = re.sub('[^a-zA-Z0-9]', '', row['check2'])
        checkvalue = df_macola1.loc[df_macola1['Phone'] == row['Provider Business Mailing Address Telephone Number'], 'check2'].values
        if len(checkvalue)>0:
            f2 = re.sub('[^a-zA-Z0-9]', '', checkvalue[0])
             #check3 = re.sub('[^a-zA-Z0-9]', '', row['fullname'])
            #check4 = re.sub('[^a-zA-Z0-9]', '', row1['fullname'])
            if jellyfish.jaro_winkler(f1,f2)>.86:
                index_lf.append(index)   
                f = open(r"C:\Users\KC\Desktop\EssilorWP\fuzzysearchvaluesBA.txt", 'a')
                f.write(str(index) + ',' + f1 + ',' + f2 + '\n')
                f.close 
print(len(index_lf))

48747
38380
11647


In [14]:
fuzzindex2 = []
fuzzdata2 = pd.read_csv(r"C:\Users\KC\Desktop\EssilorWP\fuzzysearchvaluesBA.txt", header = None)
fuzzdata2.columns = ['a','b','c']

In [15]:
fuzzdata2

,a,b,c
0,2340044,183630THSTBOULDER,183630THSTBOULDER
1,2338029,1960BETHELRDSUITE150COLUMBUS,1960BETHELRDSTE150COLUMBUS
2,2352889,32241CAMINOCAPISTRANOSTEA101SANJUANCAPISTRANO,32241CAMINOCAPISTRANOSTEA101SANJUANCAPISTRANO
3,2370861,3211ROGERSRDSTE100WAKEFOREST,3211ROGERSRDSTE100WAKEFOREST
4,2276830,3417SPECTRUMBLVDSUITE100RICHARDSON,3417SPECTRUMBLVDRICHARDSON
5,2273314,2161MAINSTDUNEDIN,2161MAINSTDUNEDIN
6,2296763,1MILLSCIR1016ONTARIO,1MILLSCIRSTE1016ONTARIO
7,2304715,7587JEFFERSONHWYBATONROUGE,7587JEFFERSONHWYBATONROUGE
8,2206469,150STRAWBERRYPLAINSRDSTEBWILLIAMSBURG,150STRAWBERRYPLAINSRDWILLIAMSBURG
9,2228581,15265THAVESANRAFAEL,15265THAVESANRAFAEL


In [16]:
for index,row in fuzzdata2.iterrows():
    if str(row.b[0:8]) == str(row.c[0:8]):
        if len(re.findall(r'\d+', row.b))>=2 & len(re.findall(r'\d+', row.c))>=2:
            if (re.findall(r'\d+', row.b)[1] == re.findall(r'\d+', row.c)[1]):
                fuzzindex2.append(row.a)
                f = open(r"C:\Users\KC\Desktop\EssilorWP\ram2fuzzyvalues-22.txt", 'a')
                f.write(str(row.a) + ',' + str(row.b) + ',' + str(row.c) + '\n')
                f.close 
        if len(re.findall(r'\d+', row.b))==1 & len(re.findall(r'\d+', row.c))==1:
            if (re.findall(r'\d+', row.b)[0] == re.findall(r'\d+', row.c)[0]):
                fuzzindex2.append(row.a)
                f = open(r"C:\Users\KC\Desktop\EssilorWP\ram2fuzzyvalues-22.txt", 'a')
                f.write(str(row.a) + ',' + str(row.b) + ',' + str(row.c) + '\n')
                f.close
                
        if len(re.findall(r'\d+', row.b))==1 & len(re.findall(r'\d+', row.c))>1:
            if (re.findall(r'\d+', row.b)[0] == re.findall(r'\d+', row.c)[0]):
                fuzzindex2.append(row.a)
                f = open(r"C:\Users\KC\Desktop\EssilorWP\ram2fuzzyvalues-22.txt", 'a')
                f.write(str(row.a) + ',' + str(row.b) + ',' + str(row.c) + '\n')
                f.close
                
        if len(re.findall(r'\d+', row.b))>1 & len(re.findall(r'\d+', row.c))==1:
            if (re.findall(r'\d+', row.b)[0] == re.findall(r'\d+', row.c)[0]):
                fuzzindex2.append(row.a)
                f = open(r"C:\Users\KC\Desktop\EssilorWP\ram2fuzzyvalues-22.txt", 'a')
                f.write(str(row.a) + ',' + str(row.b) + ',' + str(row.c) + '\n')
                f.close
print(len(fuzzindex2))

10539


In [35]:
df_macola_toenrich['flag'] = None
df_pa_enrich = pd.DataFrame()
for index1,row in df_macola_toenrich.iterrows():
    a=df_npa.loc[df_npa['Provider Business Practice Location Address Telephone Number'] == row['Practice Phone'], 'check1'].values
    b=df_npa.loc[df_npa['Provider Other Organization Name'] == row['Practice Name'], 'Practice Name'].values
    if len(b)>0:
        
        for i in range(len(a)):
            if df_
        df_macola_toenrich.loc[index1,'Provider First Line Business Practice Location Address'] = a[0]
        print(a[0])
        df_macola_toenrich.loc[index1,'flag'] = 'PA-ER'
        df_pa_enrich.append([df_macola_toenrich[df_macola_toenrich.index==index1]])
df_pa_enrich.shape

SyntaxError: invalid syntax (<ipython-input-35-55b3bd830f57>, line 9)